In [1]:
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt

In [2]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate,train_test_split,KFold

In [3]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise import get_dataset_dir

In [4]:
from surprise import accuracy

In [5]:
from surprise import SVD
from surprise import NMF
from surprise import similarities

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [7]:
import math

### import large ratings file 

In [8]:
file_path = os.path.expanduser('~/Downloads/ml-20m/ratings.csv')
reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)

In [58]:
large_ratings = pd.read_csv(file_path)

In [61]:
large_ratings.drop('timestamp',axis=1,inplace=True)

In [93]:
# avg rating for movies with more than 250 reviews
avg_rating = large_ratings.groupby('movieId').mean()\
[large_ratings.groupby('movieId').count()['userId'] > 250].reset_index()[['movieId','rating']]

In [118]:
movie_id_inner_id.raw_id = movie_id_inner_id.raw_id.astype(int)

In [120]:
avg_rating['movie_name'] = avg_rating.movieId.apply(lambda x: item_name_dict[str(x)])

avg_rating = avg_rating.merge(movie_id_inner_id, left_on='movieId', right_on='raw_id')

In [134]:
# recreate the inner id (by reseting index) which serves as the matrix row, so how we go from row to movie name
movie_dict_reduced = avg_rating.reset_index()

In [137]:
movie_dict_reduced = movie_dict_reduced[['index','movie_name','rating']]

In [141]:
movie_dict_reduced.columns = ['matrix_row','movie_name','avg_rating']

In [143]:
movie_dict_reduced['year'] = movie_dict_reduced.movie_name.apply(return_year)

/Users/Mark/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [163]:
movie_dict_reduced['avg_rating'] = movie_dict_reduced['avg_rating'].map(lambda x: '%1.2f' % x)

In [167]:
movie_dict_reduced.tail()

,matrix_row,movie_name,avg_rating,year
6137,6137,The Imitation Game (2014),3.98,2014
6138,6138,The Hunger Games: Mockingjay - Part 1 (2014),3.45,2014
6139,6139,"Theory of Everything, The (2014)",3.72,2014
6140,6140,The Hobbit: The Battle of the Five Armies (2014),3.41,2014
6141,6141,The Interview (2014),2.90,2014


In [166]:
movie_dict_reduced[['movie_name','avg_rating','year']].to_csv('movie_dict.csv')

In [149]:
np.arange(0,10).tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [151]:
sim_pca.shape

(26744, 15)

In [153]:
sim_pca_reduced.shape

(6142, 10)

In [152]:
sim_pca_reduced = sim_pca[avg_rating.inner_id.values,:10]

In [157]:
np.savetxt("movie_factors.csv", sim_pca_reduced, delimiter=",",fmt='%1.3f')

### import small ratings file 

In [ ]:
# path to dataset file
# file_path = os.path.expanduser('~/Downloads/ml-20m/ratings.csv')
file_path = os.path.expanduser('~/Downloads/ml-latest-small/ratings.csv')
reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)

In [ ]:
small_ratings = pd.read_csv(file_path)

In [9]:
item_name_dict = {}
name_item_dict = {}

# file_path = os.path.expanduser('~/Downloads/ml-latest-small/movies.csv')
file_path = os.path.expanduser('~/Downloads/ml-20m/movies.csv')

with open(file_path, "r") as csvfile:
    ff = csv.reader(csvfile)
    for row in ff:
        item_name_dict[row[0]] = row[1]
        name_item_dict[row[1]] = row[0]

In [ ]:
movieId_avg_rating = small_ratings.groupby('movieId').mean().reset_index()[['movieId','rating']]
movieId_avg_rating_dict = {row.movieId:row.rating for index,row in movieId_avg_rating.iterrows()}

In [ ]:
userId_avg_rating = small_ratings.groupby('userId').mean().reset_index()[['userId','rating']]
userId_avg_rating_dict = {row.userId:row.rating for index,row in userId_avg_rating.iterrows()}

In [ ]:
def mean_center_movie(row):
    return row['rating'] - movieId_avg_rating_dict[row['movieId']]

In [ ]:
def mean_center(row):
    return row['rating'] - userId_avg_rating_dict[row['userId']]

In [ ]:
small_ratings['rating_adj'] = small_ratings.apply(mean_center,axis=1)
# small_ratings['rating_adj'] = small_ratings.apply(lambda x: x['rating'] - userId_rating_dict[x['userId']] + 2.5)

In [ ]:
small_ratings.describe()

In [ ]:
row_to_userId = {x:y for x,y in enumerate(small_ratings.userId.unique())}
userId_to_row = {y:x for x,y in enumerate(small_ratings.userId.unique())}

In [ ]:
col_to_movieId = {x:y for x,y in enumerate(small_ratings.movieId.unique())}
movieId_to_col = {y:x for x,y in enumerate(small_ratings.movieId.unique())}

In [ ]:
X_mean =  sparse.csr_matrix((small_ratings['rating_adj'],\
                        ([userId_to_row[x] for x in small_ratings['userId']],\
                          [movieId_to_col[x] for x in small_ratings['movieId']])), shape=(671, 9066))

In [ ]:
X =  sparse.csr_matrix((small_ratings['rating'],\
                        ([userId_to_row[x] for x in small_ratings['userId']],\
                          [movieId_to_col[x] for x in small_ratings['movieId']])), shape=(671, 9066))

In [ ]:
X_mean_na = X_mean.toarray()
X_mean_na[:] = np.nan

In [ ]:
for index,row in small_ratings.iterrows():
    X_mean_na[userId_to_row[row.userId],movieId_to_col[row.movieId]] = row.rating_adj

In [ ]:
# augmented matrix

In [ ]:
# fill nan with item mean
X_array_mean_filled = X.toarray()

# column mean
col_mean = np.true_divide(X_array_mean_filled.sum(0),(X_array_mean_filled!=0).sum(0))

#Find indicies that you need to replace
inds = np.where(X_array_mean_filled == 0)

#Place column means in the indices. Align the arrays using take
X_array_mean_filled[inds] = np.take(col_mean, inds[1])

In [ ]:
# add year and rating to movie data
small_ratings.head()

In [ ]:
movie_dict = pd.DataFrame({'movie_id':small_ratings.movieId.unique()})

In [ ]:
movie_dict['matrix_row'] = movie_dict.movie_id.apply(lambda x: movieId_to_col[x])
movie_dict['movie_name'] = movie_dict.movie_id.apply(lambda x: item_name_dict[str(x)])
movie_dict['avg_rating'] = movie_dict.movie_id.apply(lambda x: movieId_avg_rating_dict[x])

In [ ]:
movie_dict.to_csv('movie_dict.csv')

In [142]:
def return_year(row):
    
    if any(i.isdigit() for i in row):
    
        year_pos = len(row.split('(')) - 1

        year = row.split('(')[year_pos].split(')')[0]

        if year.find('-'):
            return int(year[:4])
        else:
            return int(year)
    else:
        return int(1800)

In [ ]:
movie_dict['year'] = movie_dict.movie_name.apply(return_year)

### similar movies comparison

In [ ]:
def return_similar_movies(movie_vectors, movie_id, row_id):
    
    if movie_id == '':
        column = row_id
    else:
        column = movieId_to_col[movie_id]
    
    similarities = cosine_similarity(movie_vectors)
    
    sim_movies = [[item_name_dict[str(col_to_movieId[x])],movieId_avg_rating_dict[col_to_movieId[x]]]\
                  for x in np.argsort(-similarities[column])[:20] if x < 9066]
    
    return sim_movies

In [ ]:
movieId_avg_rating_dict[int(441)]

In [ ]:
dotted = matrix.dot(vector)
matrix_norms = np.linalg.norm(matrix, axis=1)
vector_norm = np.linalg.norm(vector)
matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
neighbors = np.divide(dotted, matrix_vector_norms)

### ML HW 4

In [ ]:
# coordinate ascent to build movie and user matrix factorizations
def matrix_fact(numb_factor, iterations, user_item_matrix):
    lamb = 1
    varsq = .25
    
    user_m = np.random.multivariate_normal(np.zeros(numb_factor),np.eye(numb_factor),user_item_matrix.shape[0])
    movie_m = np.random.multivariate_normal(np.zeros(numb_factor),np.eye(numb_factor),user_item_matrix.shape[1])
    first_part = lamb*varsq*np.eye(numb_factor)

    for t in range(0,iterations):

        for j in range(0,user_m.shape[0]):

            locas_nan = np.where(~np.isnan(user_item_matrix[j,:]))
            v_dot = np.dot(np.transpose(movie_m[locas_nan]),movie_m[locas_nan])
            m_v_dot = np.dot(user_item_matrix[j][locas_nan],movie_m[locas_nan])         
            user_m[j] = (np.dot(np.linalg.inv(first_part+v_dot),m_v_dot)).flatten()

        for k in range(0,movie_m.shape[0]):

            locas_nan = np.where(~np.isnan(user_item_matrix[:,k]))
            u_dot = np.dot(np.transpose(user_m[locas_nan]),user_m[locas_nan])
            m_u_dot = np.dot(user_item_matrix[:,k][locas_nan],user_m[locas_nan])
            movie_m[k] = (np.dot(np.linalg.inv(first_part+u_dot),m_u_dot)).flatten()
        
    return user_m, movie_m

In [ ]:
user_vectors, movie_vectors = matrix_fact(15,150,X_mean_na)

In [ ]:
RMSE = 0
count = 0
for index,row in small_ratings.iterrows():
    
    guess = np.dot(user_vectors[userId_to_row[row[0]]],movie_vectors[movieId_to_col[row[1]]])
    RMSE += (float(row[4])-guess)**2
    count += 1

RMSE_run = math.sqrt(RMSE/count)

In [ ]:
RMSE_run

In [ ]:
np.savetxt("movie_factors_test.csv", movie_vectors, delimiter=",",fmt='%1.4f')

In [ ]:
return_similar_movies(movie_vectors,3,'')

In [ ]:
item_name_dict

### Scikit NMF

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
model = NMF(n_components=15, init='nndsvda', random_state=0, verbose=True, solver='cd', alpha = .5)

In [ ]:
W = model.fit_transform(X.toarray())
H = model.components_.T

In [ ]:
return_similar_movies(H,1,'')

In [ ]:
model = NMF(n_components=10, init='nndsvda', random_state=0, verbose=True, solver='cd', alpha = .5)

In [ ]:
W_mean_filled_cols = model.fit_transform(X_array_mean_filled)
H_mean_filled_cols = model.components_.T

In [ ]:
return_similar_movies(H_mean_filled_cols,344,"")

In [ ]:
# filling in the column mean makes the results worse

In [ ]:
### matrix math - R.T * R to see where people who like one movie, like another

In [ ]:
X_mean_positive = (X_mean.toarray() > 0) * X_mean.toarray()

In [ ]:
X_matrix_product = np.dot(X_mean_positive.T,X_mean_positive)

In [ ]:
def return_highest_movie_in_column(matrix,movie_id):
    
    sim_movies = [item_name_dict[str(col_to_movieId[x])] for x in np.argsort(-matrix[:,movieId_to_col[movie_id]])[:20] if x < len(item_name_dict)]

    return sim_movies
    

In [ ]:
tf_idf_count_matrix = np.dot((X_mean.toarray() > 0).astype(int).T,(X_mean.toarray() != 0).astype(int))

In [ ]:
idf = np.log(9066/np.diag(tf_idf_count_matrix))

In [ ]:
tf_idf_results = tf_idf_count_matrix * idf

In [ ]:
tf_idf_results[tf_idf_results == np.inf] = 0
tf_idf_results = np.nan_to_num(tf_idf_results)

In [ ]:
return_highest_movie_in_column(X_matrix_product,1)

### surprise similarity

In [24]:
def return_similar_movies_surprise(movie_vectors, movie_id, row_id):
    
    if movie_id == '':
        column = row_id
    else:
        column = algo.trainset.to_inner_iid(movie_id)
    
    similarities = cosine_similarity(movie_vectors)
    
    sim_movies = [item_name_dict[str(algo.trainset.to_raw_iid(x))]\
                  for x in np.argsort(-similarities[column])[:20]]
    
    return sim_movies

In [10]:
file_path = os.path.expanduser('~/Downloads/ml-20m/ratings.csv')
reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=2)

In [11]:
data = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [12]:
trainset = data.build_full_trainset()                                         

In [ ]:
# pearson baseline > pearson > cosine > msd

In [13]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
# algo = KNNBaseline(sim_options=sim_options)
algo = KNNBasic(sim_options=sim_options)

algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
def return_neighbors_from_sim(algo,movie_id):
    # Retrieve inner id of the movie Toy Story

    movie_name = item_name_dict[str(movie_id)]
    print('starting_movie: {}'.format(movie_name))
    
    movie_raw_id = name_item_dict[movie_name]
    movie_inner_id = algo.trainset.to_inner_iid(movie_raw_id)

    # Retrieve inner ids of the nearest neighbors of Toy Story.
    movie_neighbors = algo.get_neighbors(movie_inner_id, k=20)

    # Convert inner ids of the neighbors into names.
    movie_neighbors = [algo.trainset.to_raw_iid(inner_id)
                           for inner_id in movie_neighbors]
    movie_neighbors = [item_name_dict[rid]
                           for rid in movie_neighbors]
    return movie_neighbors

In [ ]:
return_neighbors_from_sim(algo,1)

In [ ]:
# surprise similarity is really good but the matrix is too big too store, so trying PCA
# try PCA on sim matrix

In [ ]:
algo.compute_similarities().shape

In [15]:
from sklearn.decomposition import PCA

In [56]:
np.savetxt("ml_20m_sim_pca_matrix.csv", sim_pca, delimiter=",",fmt='%1.4f')

In [16]:
pca = PCA(n_components=15)
sim_pca = pca.fit_transform(algo.compute_similarities())
print(pca.explained_variance_ratio_.sum())  

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
0.3618260207672808


In [48]:
movie_id_inner_id = pd.DataFrame.from_dict({key:algo.trainset.to_raw_iid(key) for key in algo.trainset.all_items()}, orient='index').reset_index()

In [111]:
movie_id_inner_id.columns = ['inner_id','raw_id']

In [55]:
movie_id_inner_id.to_csv('ml_20m_sim_pca_matrix_movie_id_inner_id_dict.csv')

In [106]:
return_similar_movies_surprise(sim_pca[:,np.arange(0,12).tolist()],'364','')

['Lion King, The (1994)',
 'Aladdin (1992)',
 'Romancing the Stone (1984)',
 'Big (1988)',
 'Field of Dreams (1989)',
 'Little Mermaid, The (1989)',
 'Beauty and the Beast (1991)',
 'Star Trek IV: The Voyage Home (1986)',
 'E.T. the Extra-Terrestrial (1982)',
 "Bug's Life, A (1998)",
 'Apollo 13 (1995)',
 'Cocoon (1985)',
 'Splash (1984)',
 'Back to the Future (1985)',
 'Monsters, Inc. (2001)',
 'Dances with Wolves (1990)',
 'Hunchback of Notre Dame, The (1996)',
 'City Slickers (1991)',
 'When Harry Met Sally... (1989)',
 'Superman (1978)']

In [ ]:
return_similar_movies(movie_vectors,1,'')

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:
return_similar_movies(algo.qi,10,'')

### hypothetical user with just a 5 rating for given movie

In [ ]:
# path to dataset file
# file_path = os.path.expanduser('~/Downloads/ml-20m/ratings.csv')
file_path_adj = os.path.expanduser('~/Downloads/ml-latest-small/ratings_adj.csv')

In [ ]:
672	1	5
673	10	5
674	344	5
675	364	5

In [ ]:
data = Dataset.load_from_file(file_path_adj, reader=reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



In [ ]:
from collections import defaultdict

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

### combining vectors

In [ ]:
def plot_two_items(movie_factors,*args):
    
    for i in args:
        print('movie_name: {}'.format((item_name_dict[str(i)])))
    
    item_cols = [movieId_to_col[int(x)] for x in args]
    
#     closest_cols = np.argsort(abs(movie_factors[item_1_col]-movie_factors[item_2_col]))[:10]
#     new_matrix = movie_factors[:,closest_cols]
    
    new_matrix = movie_factors[:,:]
    
    print('max: {}'.format(new_matrix.shape[0]))
    max_orig_array = np.maximum.reduce(movie_factors[item_cols])
    new_matrix = np.append(new_matrix, np.array(max_orig_array).reshape(1,-1),axis = 0)
    
    print('min: {}'.format(new_matrix.shape[0])) 
    min_orig_array = np.minimum.reduce(movie_factors[item_cols])
    new_matrix = np.append(new_matrix, np.array(min_orig_array).reshape(1,-1),axis = 0)
    
    print('mean: {}'.format(new_matrix.shape[0]))      
    mean_of_items = np.mean(movie_factors[item_cols],axis=0)
    new_matrix = np.append(new_matrix, mean_of_items.reshape(1,-1), axis=0)
    
    return new_matrix
    
    perc = np.percentile(movie_factors, np.arange(0,101,10),axis=0)
    
    print(perc.shape)
    
    movie_1 = []
    movie_2 = []
    movie_3 = []
    
    print(new_matrix.shape)
    
    for x in range(perc.shape[1]):
        movie_1.append(np.digitize(movie_factors[item_1_col,x], perc[:,x]).flatten()[0])
        movie_2.append(np.digitize(movie_factors[item_2_col,x], perc[:,x]).flatten()[0])
        movie_3.append(np.digitize(movie_factors[item_3_col,x], perc[:,x]).flatten()[0])

    max_array = []
    for x,i in enumerate(np.maximum.reduce([movie_1,movie_2,movie_3])):
        max_array.append(np.percentile(movie_factors[:,x],(i-1)*10))
    
    print(np.array(max_array).reshape(1,-1).shape)
    
    new_matrix = np.append(new_matrix, np.array(max_array).reshape(1,-1),axis = 0)
   
    min_array = []
    for x,i in enumerate(np.minimum.reduce([movie_1,movie_2,movie_3])):
        min_array.append(np.percentile(movie_factors[:,x],(i-1)*10))
        
    new_matrix = np.append(new_matrix, np.array(min_array).reshape(1,-1),axis = 0)    
    
    mean_of_items = np.mean([movie_factors[item_1_col,:],movie_factors[item_2_col,:],movie_factors[item_3_col,:]],axis=0)
    
    new_matrix = np.append(new_matrix, mean_of_items.reshape(1,-1), axis=0)
    
    min_vector = np.minimum.reduce(movie_factors[[item_1_col,item_2_col,item_3_col]])
    
    new_matrix = np.append(new_matrix, min_vector.reshape(1,-1), axis=0)
    
    summed_vector = np.zeros(new_matrix.shape[1])
    
    for i in [item_1_col,item_2_col,item_3_col]:
        summed_vector += new_matrix[i]

    
    new_matrix = np.append(new_matrix, summed_vector.reshape(1,-1), axis=0)

#     new_matrix_sparse = sparse.csr_matrix(new_matrix)
#     new_matrix_sparse = sparse.csr_matrix(algo.qi)
    
#     similarities = cosine_similarity(new_matrix)
    
#     [[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9066])[-20:-1]]
    

    
    return new_matrix
    
#     return new_matrix,movie_factors[item_1_col],movie_factors[item_2_col],movie_factors[item_3_col],movie_1,movie_2,movie_3


# A =  np.array([[0, 1, 0, 0, 1], [0, 0, 1, 1, 1],[1, 1, 0, 1, 0]])
# A_sparse = sparse.csr_matrix(A)

# similarities = cosine_similarity(A_sparse)
# print('pairwise dense output:\n {}\n'.format(similarities))

# #also can output sparse matrices
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [ ]:
combined_vectors = plot_two_items(sim_pca,1136,2788,)

In [ ]:
combined_vectors.shape

In [ ]:
return_similar_movies(combined_vectors,'',9068)

In [ ]:
movieId_avg_rating_dict[541]

In [ ]:
return_similar_movies(combined_vectors,'',9068)

In [ ]:
conversion to percentiles and then minimum worked best for
'Casper (1995)','Toy Story (1995)','Pocahontas (1995)'

In [ ]:
combined_vectors[9069]

In [ ]:
combined_vectors[9068]

In [ ]:
name_item_dict['Toy Story (1995)']

In [ ]:
movieId_to_col[1]

In [ ]:
combined_vectors.shape

In [ ]:
for i in top_n['676']:
    print(item_name_dict[i[0]])

In [ ]:
small_ratings.userId.max()

In [ ]:
def sum_similarities(rows):
    
    min_vector = np.minimum.reduce(similarities[rows])
    
    summed_vector = np.zeros(similarities.shape[1])
    for i in rows:
        summed_vector += similarities[i]
        print(item_name_dict[str(col_to_movieId[i])])
    return summed_vector, min_vector

In [ ]:
sum_vector, min_vector = sum_similarities([143,4939])

In [ ]:
min_vector

In [ ]:
similarities[143]

In [ ]:
similarities[4939]

In [ ]:
summed_vector

In [ ]:
similarities[966][3679]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-min_vector)[:20] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-similarities_orig[966])[:20] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-similarities_sur[966])[:20] if x < 9066]

In [ ]:
from sklearn.decomposition import SparsePCA

In [ ]:
pca = SparsePCA(n_components=10)
X_sparse_PCA = pca.fit_transform(X.toarray().T)
print(pca.explained_variance_ratio_.sum())  

In [ ]:
matrix = X_sparse_PCA
vector = X_sparse_PCA[965]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-neighbors)[:10] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-similarities[965])[:10] if x < 9066]

In [ ]:
np.savetxt("movie_factors.csv", movie_factors, delimiter=",")

In [ ]:
movie_dict = pd.DataFrame({'movie_id':small_ratings.movieId.unique()})

In [ ]:
movie_dict['matrix_row'] = movie_dict.movie_id.apply(lambda x: movieId_to_col[x])
movie_dict['movie_name'] = movie_dict.movie_id.apply(lambda x: item_name_dict[str(x)])

In [ ]:
movie_dict.to_csv('movie_dict.csv')

In [ ]:
movie_factors.shape

In [ ]:
test = np.loadtxt("movie_factors.csv", delimiter=",")

In [ ]:
[test] == [movie_factors]

In [ ]:
similarities = cosine_similarity(movie_factors)

In [ ]:
similarities_sur = cosine_similarity(algo.qi)

In [ ]:
matrix = movie_factors
vector = agg_vector

In [ ]:
dotted = matrix.dot(vector)
matrix_norms = np.linalg.norm(matrix, axis=1)
vector_norm = np.linalg.norm(vector)
matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
neighbors = np.divide(dotted, matrix_vector_norms)

In [ ]:
neighbors.shape

In [ ]:
new_matrix = np.append(movie_factors, agg_vector.reshape(1,-1), axis=0)

In [ ]:
similarities = cosine_similarity(new_matrix)

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(similarities[9066])[-10:] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(-neighbors)[:10] if x < 9066]

In [ ]:
item_name_dict

In [ ]:
col_to_movieId[]

In [ ]:
small_ratings.movieId.unique()

In [ ]:
pd.DataFrame({'movie_factors':movie_factors})

In [ ]:
movie_dict.head()

In [ ]:
movie_value = ['1','100']

In [ ]:
cols = movie_dict[movie_dict.movie_id.isin(movie_value)].matrix_row.values

In [ ]:
movie_factors[417]

In [ ]:
movie_factors[2071]

In [ ]:
movie_factors[cols,:]

In [ ]:
agg_vector = np.maximum.reduce(movie_factors[cols,:])

In [ ]:
agg_vector

In [ ]:
movie_factors.shape

In [ ]:
movie_factors[1]

In [ ]:
movie_factors[100]

In [ ]:
np.maximum.reduce([movie_factors[100],movie_factors[1]])

In [ ]:
first_try,movie_1,movie_2,movie_3,movie_1_perc,movie_2_perc,movie_3_perc = plot_two_items(movie_factors,'Casper (1995)','Toy Story (1995)','Pocahontas (1995)')

In [ ]:
item_name_dict[str(col_to_movieId[72])]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[movieId_to_col[719]])[-10:] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[movieId_to_col[539]])[-10:] if x < 9066]

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[movieId_to_col[10]])[-10:] if x < 9066]

In [ ]:
# max from original arrays
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9066])[-10:] if x < 9066]

In [ ]:
# max from percentiles of original arrays
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9067])[-10:] if x < 9066]

In [ ]:
# min from percentiles of original arrays
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9068])[-10:] if x < 9066]

In [ ]:
# mean of all arrays
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9069])[-10:] if x < 9066]

In [ ]:
for key in item_name_dict:
    print('{',"""'label': "{}", 'value': {}""".format(item_name_dict[key],str(key)),'},')

In [ ]:
sample = pd.DataFrame([item_name_dict]).T.reset_index()
sample.columns = ['movieId','movie_name']
sample.to_csv('movie_test.csv')

In [ ]:
movie_dict.head()

In [ ]:
len(sample)

In [ ]:
df = sample

In [ ]:
df[df.movieId == '148'].movie_name.values[0]

In [ ]:
for i in df.movie_name.head():
    print(i)

In [ ]:
def make_dash_table(df):
    ''' Return a dash definition of an HTML table for a Pandas dataframe '''
    table = []
    for index, row in df.iterrows():
        html_row = []
        for i in range(len(row)):
            html_row.append(html.Td([row[i]]))
        table.append(html.Tr(html_row))
    return table

In [ ]:
len(item_name_dict)

In [ ]:
movie_1

In [ ]:
movie_2

In [ ]:
movie_1_perc

In [ ]:
movie_2_perc

In [ ]:
np.multiply(movie_1_perc,movie_2_perc)

In [ ]:
np.maximum.reduce([movie_1_perc,movie_2_perc])

In [ ]:
new_array = np.array()
for x,i in enumerate(np.maximum.reduce([movie_1_perc,movie_2_perc])):
    new_array.append(np.percentile(movie_factors[:,x],(i-1)*10))

In [ ]:
np.array(new_array).shape

In [ ]:
[0,1]*5

In [ ]:
new_factors = [0,1]*5

In [ ]:
movie_1[]

In [ ]:
for i in range(movie_factors.shape[1]):
    print(movie_factors[:,i].mean(),movie_factors[:,i].min(),movie_factors[:,i].max())

In [ ]:
perc[:,2]

In [ ]:
perc.shape

In [ ]:
movie_1=[]
movie_2=[]
for x in perc.shape[1]:
    movie_1.append(np.digitize(movie_factors, perc[:,x]))

In [ ]:
np.arange(0,101,10)

In [ ]:
perc = np.percentile(movie_factors[:,0], np.arange(0,101,10))
indices = np.digitize(movie_factors[:,0], perc)

In [ ]:
movie_factors.shape

In [ ]:
perc = np.percentile(movie_factors, np.arange(0,101,10),axis=0)
perc

In [ ]:
movie_factors[0,:].shape

In [ ]:
indices = np.digitize(movie_factors[0,0], perc[:,0])

In [ ]:
movie_factors[0,:]

In [ ]:
perc[:,0]

In [ ]:
indices

In [ ]:
perc.shape

In [ ]:
indices[1000:]

In [ ]:
movie_1[0]

In [ ]:
movie_factors[:,0].min()

In [ ]:
plt.hist(movie_factors[:,0])

In [ ]:
raw_id = name_item_dict['Clueless (1995)']

inner_id = algo.trainset.to_inner_iid(raw_id)

In [ ]:
[[x,item_name_dict[str(col_to_movieId[x])]] for x in np.argsort(first_try[9066])[-20:-1]]

In [ ]:
first_try[9066]

In [ ]:
item_name_dict

In [ ]:
len(movieId_to_col)

In [ ]:
data

In [ ]:
data = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
trainset = data.build_full_trainset()                                         
algo.train(trainset)

In [ ]:
algo = SVD(n_factors = 10)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)

In [ ]:
algo.qi.shape

In [ ]:
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
new_matrix_sparse = sparse.csr_matrix(algo.qi)

similarities = cosine_similarity(new_matrix_sparse)

In [ ]:
similarities.shape

In [ ]:
np.argsort(-similarities[988]) 

In [ ]:
# raw_id = name_item_dict['Ponyo (Gake no ue no Ponyo) (2008)']
# raw_id = name_item_dict['Clueless (1995)']
raw_id = name_item_dict['Toy Story (1995)']

inner_id = algo.trainset.to_inner_iid(raw_id)

In [ ]:
inner_id

In [ ]:
[[x,item_name_dict[algo.trainset.to_raw_iid(x)]] for x in np.argsort(-similarities[inner_id])[:10]]

In [ ]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

In [ ]:
# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_item_dict['Clueless (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = [algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors]
toy_story_neighbors = [item_name_dict[rid]
                       for rid in toy_story_neighbors]

In [ ]:
toy_story_neighbors

In [ ]:
kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

In [ ]:
algo.compute_similarities().shape

In [ ]:
len(item_name_dict)

In [ ]:
# plt.plot(algo.qi[0])
# plt.plot(algo.qi[1])
plt.plot(algo.qi[0]-algo.qi[1])

In [ ]:
closest_cols = np.argsort(abs(algo.qi[0]-algo.qi[1]))[:50]

In [ ]:
algo.qi[0,closest_cols],algo.qi[1,closest_cols]

In [ ]:
np.mean([algo.qi[0,closest_cols],algo.qi[1,closest_cols]],axis=0)

In [ ]:
new_matrix = algo.qi[:,closest_cols]

In [ ]:
new_matrix.shape

In [ ]:
item_1_raw_id = name_item_dict[item_1]
item_2_raw_id = name_item_dict[item_2]

item_1_inner_id = algo.trainset.to_inner_iid(item_1_raw_id)
item_2_inner_id = algo.trainset.to_inner_iid(item_2_raw_id)


In [ ]:
new_matrix.shape

In [ ]:
mean_of_items.reshape(1,-1).shape

In [ ]:
new_matrix = algo.qi[:,closest_cols]

mean_of_items = np.mean([algo.qi[0,closest_cols],algo.qi[1,closest_cols]],axis=0)

np.append(new_matrix, mean_of_items.reshape(1,-1), axis=0)

In [ ]:
def plot_two_items(algo,item_1,item_2):
    item_1_raw_id = name_item_dict[item_1]
    item_2_raw_id = name_item_dict[item_2]

    item_1_inner_id = algo.trainset.to_inner_iid(item_1_raw_id)
    item_2_inner_id = algo.trainset.to_inner_iid(item_2_raw_id)
    
    closest_cols = np.argsort(abs(algo.qi[item_1_inner_id]-algo.qi[item_2_inner_id]))[:100]
    new_matrix = algo.qi[:,closest_cols]
    
    mean_of_items = np.mean([algo.qi[0,closest_cols],algo.qi[1,closest_cols]],axis=0)
    
#     new_matrix = np.append(new_matrix, mean_of_items.reshape(1,-1), axis=0)
    
#     new_matrix_sparse = sparse.csr_matrix(new_matrix)
    new_matrix_sparse = sparse.csr_matrix(algo.qi)
    
    similarities = cosine_similarity(new_matrix_sparse)
        
    return similarities


# A =  np.array([[0, 1, 0, 0, 1], [0, 0, 1, 1, 1],[1, 1, 0, 1, 0]])
# A_sparse = sparse.csr_matrix(A)

# similarities = cosine_similarity(A_sparse)
# print('pairwise dense output:\n {}\n'.format(similarities))

# #also can output sparse matrices
# similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [ ]:
first_try = plot_two_items(algo,'Leaving Las Vegas (1995)','Clueless (1995)')

In [ ]:
raw_id = name_item_dict['Clueless (1995)']

inner_id = algo.trainset.to_inner_iid(raw_id)

In [ ]:
[[x,item_name_dict[algo.trainset.to_raw_iid(x)]] for x in np.argsort(first_try[988])[-10:]]

In [ ]:
algo.qi.shape

In [ ]:
abs(algo.qi[988] - algo.qi[988]).sum()

In [ ]:
algo.qi[0]

In [ ]:
new_matrix_sparse = sparse.csr_matrix(algo.qi)

similarities = cosine_similarity(new_matrix_sparse)

In [ ]:
np.argsort(-similarities[988]) 

In [ ]:
raw_id = name_item_dict['Ponyo (Gake no ue no Ponyo) (2008)']

inner_id = algo.trainset.to_inner_iid(raw_id)

In [ ]:
inner_id

In [ ]:
[[x,item_name_dict[algo.trainset.to_raw_iid(x)]] for x in np.argsort(-similarities[1503])[:10]]

In [ ]:
algo.trainset.to_inner_iid('1')

In [ ]:
item_name_dict

In [ ]:
first_try.shape

In [ ]:
item_name_dict[algo.trainset.to_raw_iid(988)]

In [ ]:
first_try[-1]

In [ ]:
first_try

In [ ]:
item_name_dict[algo.trainset.to_raw_iid[np.argsort(first_try[inner_id])[:10][0]]]

In [ ]:
algo.trainset.to_raw_iid(7078)

In [ ]:
first_try.shape

In [ ]:
algo.qia

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=10)
pca.fit(algo.qi)
print(pca.explained_variance_ratio_.sum())  


In [ ]:
algo.trainset.to_raw_iid(6065)

In [ ]:
# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_item_dict['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = [algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors]
toy_story_neighbors = [item_name_dict[rid]
                       for rid in toy_story_neighbors]

In [ ]:
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)


In [ ]:
'480': 'Jurassic Park (1993)',

In [ ]:
toy_story_neighbors

In [ ]:
toy_story_inner_id

In [ ]:
toy_story_raw_id

In [ ]:
name_item_dict

In [ ]:
small_ratings.head()

In [ ]:
X = sparse